<a href="https://colab.research.google.com/github/samsoe/mpg_geo/blob/wide/Test_CORS_and_Local_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import pandas as pd
import geopandas as gpd
from sklearn.metrics import r2_score
import numpy as np

# Source Data

In [2]:
# Define data sources (URLs)
src = ["https://storage.googleapis.com/mpg_gps/emlid/testing/240208/green.csv",
       "https://storage.googleapis.com/mpg_gps/emlid/testing/240209/green.csv",
       "https://storage.googleapis.com/mpg_gps/emlid/testing/240208/black.csv",
       "https://storage.googleapis.com/mpg_gps/emlid/testing/240209/black.csv"]

# Wrangle

In [3]:
# Load and concatenate datasets
df_combined = pd.concat(pd.read_csv(url) for url in src)

In [4]:
# Wrangle data, create GeoDataFrame, project, and extract Easting/Northing
def wrangle_data(df):
    # Split 'Name' into 'rover', 'location', and 'measurement' columns
    df["rover"] = df["Name"].str.split("_", expand=True)[0]
    df["location"] = (
        df["Name"].str.split("_", expand=True)[1].str.split("-", expand=True)[0]
    )
    df["measurement"] = (
        df["Name"].str.split("_", expand=True)[1].str.split("-", expand=True)[1]
    )

    # Convert 'Averaging end' to datetime and create a 'date' column
    df["Averaging end"] = pd.to_datetime(df["Averaging end"])
    df["date"] = df["Averaging end"].dt.strftime("%Y-%m-%d")

    # Create a GeoDataFrame with a geometry column
    geometry = gpd.points_from_xy(df['Longitude'], df['Latitude'])
    df = gpd.GeoDataFrame(df, geometry=geometry)

    # Set the initial CRS (assuming WGS84)
    df.crs = 'EPSG:4326'

    # Project to WGS 84 / UTM zone 12N
    df = df.to_crs('EPSG:32611')

    # Extract Easting and Northing
    df['Easting'] = df.geometry.x
    df['Northing'] = df.geometry.y

    return df

df_wrangle = wrangle_data(df_combined)

In [5]:
# prompt: average Easting, Northing and Ellipsoidal height grouped by rover, location and date and then ungroup
df_avg = df_wrangle.groupby([
    'rover',
    'location',
    'date'
    ])[
        ['Easting',
         'Northing',
         'Ellipsoidal height']
        ].mean().reset_index()

In [ ]:
# prompt: Using dataframe df_avg: pivot green and black to a wider dataframe with columns like Easting_green, Easting_black, etc

df_avg_wide = df_avg.pivot(index=['location', 'date'], columns='rover', values=['Easting', 'Northing', 'Ellipsoidal height']).reset_index()
df_avg_wide.columns = [col[0] + '_' + col[1] if not pd.isnull(col[1]) else col[0] for col in df_avg_wide.columns]
df_avg_wide = df_avg_wide.rename(columns={'location_': 'location', 'date_': 'date'})

# Rename

In [6]:
# Separate data, and prepare for modeling
select_variables = ["Easting", "Northing", "Ellipsoidal height"]

# Separate data, and prepare for modeling
black_data = df_avg[df_avg["rover"] == "black"].sort_values(by="location")
green_data = df_avg[df_avg["rover"] == "green"].sort_values(by="location")

# Prepare Easting, Northing, Ellipsoidal height data for modeling
black_select = black_data[select_variables].reset_index(drop=True)
black_select.columns = ["Easting_black",
                        "Northing_black",
                        "Ellipsoidal height_black"
                        ]
green_select = green_data[select_variables].reset_index(drop=True)
green_select.columns = ["Easting_green",
                        "Northing_green",
                        "Ellipsoidal height_green"
                        ]

# Analysis

## Generalized Linear Model

In [ ]:
import statsmodels.api as sm

In [ ]:
def fit_glm(y, X):
  """
  Fits a generalized linear model using the given independent and dependent variables.

  Args:
    independent_variable: A Pandas Series or NumPy array containing the independent variable data.
    dependent_variable: A Pandas Series or NumPy array containing the dependent variable data.

  Returns:
    A glm_model containing the fitted GLM model and the results of the fit.
  """
  glm_model = sm.GLM(y, X, family=sm.families.Gaussian())
  glm_results = glm_model.fit()

  return glm_model, glm_results

In [ ]:
results = {}

results["easting"] = fit_glm(black_select["Easting_black"],
                             green_select["Easting_green"])

results["northing"] = fit_glm(black_select["Northing_black"],
                              green_select["Northing_green"])

results["height"] = fit_glm(black_select["Ellipsoidal height_black"],
                            green_select["Ellipsoidal height_green"])

print(results['easting'][1].summary(), '\n\n')
print(results['northing'][1].summary(), '\n\n')
print(results['height'][1].summary(), '\n\n')

                 Generalized Linear Model Regression Results                  
Dep. Variable:          Easting_black   No. Observations:                   20
Model:                            GLM   Df Residuals:                       19
Model Family:                Gaussian   Df Model:                            0
Link Function:               Identity   Scale:                      0.00077818
Method:                          IRLS   Log-Likelihood:                 43.720
Date:                Wed, 14 Feb 2024   Deviance:                     0.014786
Time:                        17:36:46   Pearson chi2:                   0.0148
No. Iterations:                     3   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Easting_green     1.0000   8.57e-09   1.17e+08

In [ ]:
# Extract residuals
residuals = results['easting'][1].resid_pearson

# Calculate RMSE
rmse = np.sqrt(np.mean(residuals**2))
print(rmse, '\n')

# Extract coefficient for green
print(results['easting'][1].params)

0.0271896193914655 

Easting_green    1.0
dtype: float64


In [ ]:
# Extract residuals
residuals = results['northing'][1].resid_pearson

# Calculate RMSE
rmse = np.sqrt(np.mean(residuals**2))
print(rmse, '\n')

# Extract coefficient for green
print(results['northing'][1].params)

0.027646803851383295 

Northing_green    1.0
dtype: float64


In [ ]:
# Extract residuals
residuals = results['height'][1].resid_pearson

# Calculate RMSE
rmse = np.sqrt(np.mean(residuals**2))
print(rmse, '\n')

# Extract coefficient for green
print(results['height'][1].params)

0.030097011951032666 

Ellipsoidal height_green    0.999979
dtype: float64
